In [2]:
import random
class sent:
    negative="negative"
    positive="positive"
    neutral="neutral"

class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()
    
    def get_sentiment(self):
        if self.score<=2:
            return sent.negative
        elif self.score==3:
            return sent.neutral
        else:
            return sent.positive
        
class reviewcontainer:
    def __init__(self,reviews):
        self.reviews=reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evendist(self):
        neg=list(filter(lambda x:x.sentiment ==sent.negative,self.reviews))
        pos=list(filter(lambda x:x.sentiment==sent.positive,self.reviews))
        pos_shrunk=pos[0:len(neg)]
        self.reviews=neg+pos_shrunk
        random.shuffle(self.reviews)
        

In [8]:
import json

file_name='/Users/rahuld/Desktop/senti project/dataset/Books_small_10000.json'

reviews=[]
with open(file_name) as f:
    for line in f:
        review=json.loads(line)
        reviews.append(Review(review["reviewText"],review["overall"]))
    
reviews[5].text



'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [9]:
from sklearn.model_selection import train_test_split

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
training,test=train_test_split(reviews,test_size=0.33,random_state=42)

train_container=reviewcontainer(training)
train_container.evendist()
test_container=reviewcontainer(test)
test_container.evendist()

print(len(test_container.reviews))



416


In [38]:
train_x=train_container.get_text()
train_y=train_container.get_sentiment()

test_x=test_container.get_text()
test_y=test_container.get_sentiment()

train_y.count(sent.positive)

436

In [39]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer=TfidfVectorizer()
#vectorizer.fit_transform(train_x)

train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.transform(test_x)
print(train_x_vectors[0])
print(train_x_vectors[0].toarray())


  (0, 5419)	0.08530178322241727
  (0, 7976)	0.029000106915426836
  (0, 3177)	0.03735714942220534
  (0, 239)	0.15312914712021608
  (0, 5795)	0.10226393282077076
  (0, 7925)	0.038461336526862455
  (0, 6992)	0.15312914712021608
  (0, 2577)	0.15312914712021608
  (0, 447)	0.10814643357056843
  (0, 3174)	0.14435808025114097
  (0, 2634)	0.12314067142045099
  (0, 2887)	0.14435808025114097
  (0, 5549)	0.11534199522164675
  (0, 7124)	0.12314067142045099
  (0, 569)	0.051154836427568594
  (0, 2069)	0.1381349092703335
  (0, 4264)	0.07042333914303786
  (0, 4963)	0.15312914712021608
  (0, 4982)	0.05297543379467425
  (0, 5408)	0.03996921656069994
  (0, 1895)	0.1381349092703335
  (0, 5224)	0.12499612807224808
  (0, 8627)	0.13021500449844683
  (0, 7951)	0.05603045980386992
  (0, 1805)	0.10125760660496426
  :	:
  (0, 8046)	0.11831360499515031
  (0, 4722)	0.14014736483637918
  (0, 2658)	0.14559347613971493
  (0, 3452)	0.1512201239650713
  (0, 8036)	0.05944623591481769
  (0, 1181)	0.1699404029983885
  (0, 

In [40]:
###linear svm

In [41]:
from sklearn import svm

In [42]:
clf_svm=svm.SVC(kernel="linear")

clf_svm.fit(train_x_vectors,train_y)

print(test_x[3],"\n")

print(train_x[8],"\n")


print(clf_svm.predict(test_x_vectors[0]))
print(clf_svm.predict(train_x_vectors[0]))


Some people might be able to follow the diet guide lines in this book but I could not.  Requires too many food with which I am not familiar and/or don't like. 

haven't had the opportunity to open this book.  As it turns out, I have no use for this particular book. can I get a refund. 

['positive']
['positive']


In [43]:

#decision tree classifier

from sklearn.tree import DecisionTreeClassifier

clf_dec=DecisionTreeClassifier()
clf_dec.fit(train_x_vectors,train_y)

clf_dec.predict(test_x_vectors[0])

array(['positive'], dtype='<U8')

In [44]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

clf_gnb = GaussianNB()

clf_gnb.fit(train_x_vectors.toarray(), train_y)


prediction = clf_gnb.predict(test_x_vectors[0].toarray())
print(prediction)

['positive']


In [45]:
from sklearn.linear_model import LogisticRegression

clf_log=LogisticRegression()
clf_log.fit(train_x_vectors,train_y)
clf_log.predict(test_x_vectors[0])

array(['positive'], dtype='<U8')

# evaluation


In [46]:
print(clf_svm.score(test_x_vectors,test_y))

0.8076923076923077


In [47]:
from sklearn.metrics import f1_score

f1_score(test_y,clf_svm.predict(test_x_vectors),average=None,labels=[sent.positive,sent.neutral,sent.negative])

#f1_score(test_y,clf_gnb.predict(test_x_vectors.toarray()),average=None,labels=[sent.positive,sent.neutral,sent.negative])





/Users/rahuld/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


array([0.80582524, 0.        , 0.80952381])

# tuning our model

In [48]:
from sklearn.model_selection import GridSearchCV

parameters={'kernel':('linear','rbf'),'C':(1,4,8,16,32,64)}

svc=svm.SVC()

clf=GridSearchCV(svc,parameters,cv=6)
clf.fit(train_x_vectors,train_y)


print(clf)
 


GridSearchCV(cv=6, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32, 64),
                         'kernel': ('linear', 'rbf')})


In [49]:
clf.score(test_x_vectors,test_y)

0.8076923076923077

In [50]:
import pickle
with open('/Users/rahuld/Downloads/telangana_model/review_guesser.pkl','wb')as f:
    pickle.dump(clf,f)
with open('/Users/rahuld/Downloads/telangana_model/review_guesser.pkl','rb')as f:
    loaded_clf=pickle.load(f)


In [51]:
print(test_x[8])

loaded_clf.predict(test_x_vectors[8])

I have no idea how I found this book but I suspect I picked it up through one of the online communities I follow. From the very first chapter I knew I'd made a mistake. You can save yourself time and money by reading the teaser chapters or perhaps you'll decided it's for you. The h interviews for a job with the world famous, reclusive author, who has come to live in her small town to get away from the press. She's up against her 2 best friends, who never said anything to her. It's one of the most immature things I've ever read. I felt like the story should have been set in high school. The way the H and h interact throughout the book, is like 2 teenagers, who alternate between talking and not talking to each other. I'm never a fan of a character who is in another relationship whilst the story is developing between the leads. We read about her having sex with her boyfriend near the beginning and the relationship continues through a large part of the book. It was tacky and verged on chea

array(['negative'], dtype='<U8')